In [ ]:
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()  # this loads variables from .env into the environment

groq_api_key = os.getenv("GROQ_API_KEY")

In [ ]:

llm = ChatGroq(
    temperature=0,
    groq_api_key=os.getenv("GROQ_API_KEY"),
    model_name="llama3-8b-8192"
)

response = llm.invoke([
    HumanMessage(content="Who was the first person to land on the moon?")
])

print(response.content)

In [ ]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://careers.nike.com/sr-manager-product-management-inventory-flow-foundation-itc/job/R-66229")
page_data = loader.load().pop().page_content
print(page_data)

In [ ]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### TASK:
    Extract job postings in strict JSON format.
    
    Each object should include: `role`, `experience`, `skills`, and `description`.

    Respond with **only valid JSON**. Do not include any commentary, explanations, or formatting. No Markdown, no bullet points, no backticks.

    JSON only:
    """
)



chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data':page_data})
print(res.content)

In [ ]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

In [ ]:
import pandas as pd

df = pd.read_csv("techstack_portfolios.csv")
df

In [ ]:
import chromadb
import uuid

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name='portfolio')

if not collection.count():
    for _, row in df.iterrows():
            collection.add(documents=row["Techstack"],
                            metadatas={"links": row["Links"]},
                            ids=[str(uuid.uuid4())])

In [ ]:
print("Total docs:", collection.count())
all_docs = collection.get(include=["documents", "metadatas"])
for doc, meta in zip(all_docs['documents'], all_docs['metadatas']):
    print(f"- {doc} -> {meta}")


In [ ]:
results = collection.query(query_texts=["Python"], n_results=3)
metadatas = results.get("metadatas", [])

for meta in metadatas:
    print(meta)


In [ ]:
job = json_res
job['skills']

In [ ]:
prompt_email = PromptTemplate.from_template(
    """
    ### JOB DESCRIPTION:
    {job_description}

    ### INSTRUCTION:
    You are Mohan, a Business Development Executive at AtliQ — an AI and software consulting firm that helps companies scale efficiently through intelligent automation, data systems, and digital solutions.

    Your job is to write a concise, engaging cold email to the client regarding the job description above, clearly articulating how AtliQ’s expertise aligns with their needs.

    In the second half of the email, naturally highlight relevant projects from AtliQ’s portfolio (choose from the following links: {link_list}). Do not just list them or format them like bullet points. Instead, weave them into the narrative by briefly describing what was done, the impact, and hyperlinking the project title.

    You are not allowed to add a preamble or any explanation before or after the email. Return only the email.

    ###EMAIL:
    """
)


chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)